In [22]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from pathlib import Path
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# Loading Training Data

In [26]:
# create generator
datagen = ImageDataGenerator()
# prepare an iterators for each dataset
batch_size = 64

test_datagen = ImageDataGenerator(rescale=1./255) # this is the augmentation configuration we will use for testing:
# only rescaling
train_datagen = datagen.flow_from_directory('chest_xray/train/',
                                       target_size=(256, 256),
                                       batch_size= batch_size,
                                       class_mode='binary')
val_datagen = datagen.flow_from_directory('chest_xray/val/',
                                     target_size=(256, 256),
                                     batch_size= batch_size,
                                     class_mode='binary')

# confirm the iterator works
batchX, batchy = train_datagen.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Batch shape=(64, 256, 256, 3), min=0.000, max=255.000


# Setting Up NN

In [35]:
model = Sequential()
model.add(Conv2D(32,
                 (3, 3),
                 padding = 'same',
                 activation = 'relu',
                 input_shape=(256, 256,3)))

model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(512, activation="relu"))

model.add(Dropout(0.50))

model.add(Dense(1, activation="sigmoid"))

model.compile(
    loss='binary_crossentropy',
    optimizer='rmsprop', 
    metrics=['accuracy'] 
)

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 256, 256, 32)      896       
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 254, 254, 32)      9248      
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 254, 254, 64)      18496     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 127, 127, 64)      0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 127, 127, 64)      36928     
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 125, 125, 64)     

# Training the Model

In [36]:
model.fit_generator(
        train_datagen,
        steps_per_epoch=2000 // batch_size, #steps per epoch = number images
        epochs=1,
        validation_data=val_datagen,
        validation_steps=800 // batch_size)

Epoch 1/1
31/31 [==============================] - 389s 13s/step - loss: 458.2596 - accuracy: 0.7141 - val_loss: 1.1732 - val_accuracy: 0.6875